In [59]:
# Install, import libraries
import sys
!{sys.executable} -m pip install python-dotenv pandas pydap geopandas

import geopandas as gpd
import pandas as pd
from pydap.client import open_url
from pydap.cas.urs import setup_session

import xarray as xr
import requests



%load_ext dotenv
%dotenv
from os import environ

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [60]:
datasets = [ ## TODO automate the consturction of this list
    "2020/oco2_LtCO2_200101_B9003r_200204190325s.nc4",
    "2020/oco2_LtCO2_200102_B9003r_200204190440s.nc4",
    "2020/oco2_LtCO2_200103_B9003r_200204190754s.nc4",
    "2020/oco2_LtCO2_200108_B9003r_200204190946s.nc4",
    "2020/oco2_LtCO2_200109_B9003r_200204191010s.nc4",
    "2020/oco2_LtCO2_200110_B9003r_200204191136s.nc4",
    "2020/oco2_LtCO2_200111_B9003r_200204191415s.nc4",
    "2020/oco2_LtCO2_200112_B9003r_200204191626s.nc4",
    "2020/oco2_LtCO2_200113_B9003r_200204191752s.nc4",
    "2020/oco2_LtCO2_200114_B9003r_200204192011s.nc4",
    "2020/oco2_LtCO2_200115_B9003r_200204192230s.nc4",
    "2020/oco2_LtCO2_200116_B9003r_200204192449s.nc4",
    "2020/oco2_LtCO2_200117_B9003r_200204192650s.nc4",
    "2020/oco2_LtCO2_200118_B9003r_200204192849s.nc4",
    "2020/oco2_LtCO2_200119_B9003r_200204193103s.nc4",
    "2020/oco2_LtCO2_200120_B9003r_200204193223s.nc4",
    "2020/oco2_LtCO2_200121_B9003r_200204193452s.nc4"
]

In [45]:
## Based on code from http://xarray.pydata.org/en/stable/io.html
def read_datasets(files, dim, transform_func=None):
    base_url = "https://oco2.gesdisc.eosdis.nasa.gov/opendap/OCO2_L2_Lite_FP.9r/"
    session = setup_session(environ.get("EARTHDATA_USER"), environ.get("EARTHDATA_PASS"), check_url=base_url+files[0])
    def process_one_path(path):
        print(f"Loading dataset... {path}")
        # use a context manager, to ensure the file gets closed after use
        with xr.backends.PydapDataStore.open(dataset_url, session=session) as store:
            with xr.open_dataset(store) as ds:
                # transform_func should do some sort of selection or aggregation
                # aggregation
                if transform_func is not None:
                    ds = transform_func(ds)
                ds.load()
                return ds

    paths = sorted(files) # TODO use Glob
    datasets = [process_one_path(base_url+p) for p in paths]
    
    return xr.concat(datasets, dim)

# Method to reshape one dataset
def reshape_dataset(ds):
    return (ds[[ 'sounding_id', 'latitude', 'longitude', 'time', 'xco2' ]]
               .set_index(sounding_id=['latitude', 'longitude', 'time']))

# here we suppose we only care about the combined mean of each file;
# you might also use indexing operations like .sel to subset datasets
combined = read_datasets(datasets, dim='sounding_id', transform_func=reshape_dataset)

print("Done.")

Loading dataset... https://oco2.gesdisc.eosdis.nasa.gov/opendap/OCO2_L2_Lite_FP.9r/2020/oco2_LtCO2_200101_B9003r_200204190325s.nc4
Loading dataset... https://oco2.gesdisc.eosdis.nasa.gov/opendap/OCO2_L2_Lite_FP.9r/2020/oco2_LtCO2_200102_B9003r_200204190440s.nc4
Loading dataset... https://oco2.gesdisc.eosdis.nasa.gov/opendap/OCO2_L2_Lite_FP.9r/2020/oco2_LtCO2_200103_B9003r_200204190754s.nc4
Done.


In [57]:
unc = [-79.0558,35.9132]
radius = 5
result = None

try:
    result = combined.sel(
        longitude=slice(unc[0]-radius, unc[0]+radius),
        latitude=slice(unc[1]-radius,unc[1]+radius))
except KeyError as e:
    result = "Not found."

print(result)

Not found.
